In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define a custom neural network with one fully connected layer
class FullyConnectedRegressor(nn.Module):
    def __init__(self, input_size, output_size):
        super(FullyConnectedRegressor, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = self.fc(x)
        return x

# Function to initialize weights within the range (0, 1)
def initialize_weights(model):
    for param in model.parameters():
        nn.init.uniform_(param.data, a=0, b=1)

# Function to update weights within the range (0, 1)
def update_weights(model, learning_rate):
    for param in model.parameters():
        noise = torch.empty_like(param.data).uniform_(-0.1, 0.1) * learning_rate
        param.data.add_(noise)
        param.data.clamp_(0, 1)



In [3]:

# Generate synthetic data for training
N = 1000
F = 10
data = torch.randn(N, F)
true_weights = torch.rand(F, 1)  # True weights in the range (0, 1)
noise = torch.randn(N, 1) * 0.1
regression_values = torch.mm(data, true_weights) + noise

# Create a DataLoader for the training dataset
dataset = TensorDataset(data, regression_values)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define input and output sizes
input_size = F  # Example input size
output_size = 1  # Example output size

# Create an instance of the FullyConnectedRegressor model
model = FullyConnectedRegressor(input_size, output_size)

# Initialize weights within the range (0, 1)
initialize_weights(model)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)



In [4]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader, 0):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update weights within the range (0, 1)
        update_weights(model, learning_rate=0.01)

        # Print statistics
        running_loss += loss.item()
        
    print(f"Epoch [{epoch + 1}/{num_epochs}] with Loss: {running_loss/ len(train_loader):.4f}")

print("Training finished")

Epoch [1/10] with Loss: 2.2116
Epoch [2/10] with Loss: 0.6271
Epoch [3/10] with Loss: 0.1806
Epoch [4/10] with Loss: 0.0591
Epoch [5/10] with Loss: 0.0248
Epoch [6/10] with Loss: 0.0138
Epoch [7/10] with Loss: 0.0113
Epoch [8/10] with Loss: 0.0104
Epoch [9/10] with Loss: 0.0101
Epoch [10/10] with Loss: 0.0101
Training finished


In [5]:
model.state_dict()

OrderedDict([('fc.weight',
              tensor([[0.3657, 0.8729, 0.8560, 0.0458, 0.3583, 0.8094, 0.8773, 0.5967, 0.4811,
                       0.5718]])),
             ('fc.bias', tensor([0.0042]))])

In [6]:
# Generate inference data (replace this with your actual inference data)
inference_data = torch.randn(10, 10)  # Example inference data

# Perform inference
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    # Forward pass to get predictions
    predictions = model(inference_data)

# Clip predictions within the range [0, 100]
clipped_predictions = torch.clamp(predictions, 0, 100)

print("Clipped Predictions within the range [0, 100]:\n", clipped_predictions)

Clipped Predictions within the range [0, 100]:
 tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.4382],
        [0.0000],
        [1.5396],
        [0.0000],
        [0.0000],
        [0.0000]])
